In [1]:
import numpy as np
import os
import glob
import pandas as pd
from statsmodels.sandbox.stats.multicomp import multipletests
import numpy as np
from subprocess import call

In [2]:
dir_pre = "/mnt/hpc/home/xuxinran/DirectSeq/data/zhaolin_240206/240201-zhaolin-RNA-merge/v0.7.2/tss/nano_merge"

output_path = f"{dir_pre}_signal_tssqtl.csv"
output_path_all = f"{dir_pre}_tss_SNP.csv"

In [3]:
files = glob.glob(f'{dir_pre}_haplotype_chr*_tmp.csv')
dfs = [pd.read_csv(file) for file in files]
df = pd.concat(dfs, ignore_index=True)
df = df.sort_values(by=['chrom', 'snp_pos_1base'])
df = df.reset_index(drop=True)

In [4]:
df

,chrom,strand,snp_pos_1base,rsID,A1,A2,TSS_id_l,EAF,A1_TSS1,A2_TSS1,A1_TSS2,A2_TSS2,p_value,beta,SE
0,chr1,+,787399,rs2905055,G,T,"p1@ENST00000441765,p1@ENST00000449005,0.5693,o...",0.241900,6.0,14.0,2.0,9.0,0.675719,0.656780,0.921524
1,chr1,-,935833,rs3128115,G,C,"p@chr1:936069..936101,-,0.1384,other",0.115000,6.0,1.0,2.0,5.0,0.102564,2.708050,1.366260
2,chr1,+,949608,rs1921,A,G,"p1@ISG15,0.2477,other",0.237400,7881.0,19052.0,1174.0,2848.0,0.940758,0.003483,0.037179
3,chr1,+,949831,rs116002608,T,C,"p1@ISG15,0.2477,other",0.038460,1085.0,25869.0,256.0,5907.0,0.641537,-0.032749,0.070964
4,chr1,+,949889,rs3829742,A,G,"p1@ISG15,0.2477,other",0.038460,1139.0,25451.0,269.0,5896.0,0.780430,-0.019286,0.069313
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13155,chr9,+,140167985,rs28740782,T,A,"p@chr9:140167698..140167741,+,0.1704,other",0.476000,48.0,63.0,118.0,127.0,0.422971,-0.198431,0.230337
13156,chr9,+,140175422,rs117569577,T,C,"p7@C9orf167,0.2080,other",0.074260,3.0,19.0,6.0,64.0,0.442185,0.521297,0.753828
13157,chr9,+,140176814,rs112239920,A,G,"p@chr9:140176184..140176195,+,0.3062,other",0.261400,14.0,60.0,63.0,153.0,0.094642,-0.567984,0.332422
13158,chr9,+,140446707,rs149264137,A,G,"p1@MRPL41,0.3054,other",0.009709,48.0,2183.0,15.0,489.0,0.252691,-0.332942,0.300005


In [5]:
p_adjusted = multipletests(df["p_value"], method='fdr_bh')
df['fdr'] = p_adjusted[1]
print("pvalue小于0.05的个数：", len(df[df['p_value']<0.05]))
print("pvalue小于0.01的个数：", len(df[df['p_value']<0.01]))
print("fdr小于0.1的个数：", len(df[df['fdr']<0.1]))

pvalue小于0.05的个数： 584
pvalue小于0.01的个数： 186
fdr小于0.1的个数： 46


In [6]:
result_df = df[df['p_value']<0.05]
result_df.to_csv(output_path, index=False)
df.to_csv(output_path_all, index=False)

call(f"rm {dir_pre}_haplotype_chr*_tmp.csv {dir_pre}_overlap.bed", shell=True)

0